In [1]:
import json
import os.path
import time
import pandas as pd
import glob
import os
import polars as pl

import sqlalchemy

In [2]:
df1 = pl.read_csv('bfmoalive108_protocol.tsv', separator='\t')
df1

plate_name,well_position,cbk_batch,comp_vol_nL,stock_conc_mM
str,str,str,f64,str
"""bfmoalive108_P…","""A01""","""backfill""",30.0,"""100%"""
"""bfmoalive108_P…","""A02""","""BJ1894267""",10.0,"""10"""
"""bfmoalive108_P…","""A02""","""backfill""",20.0,"""100%"""
"""bfmoalive108_P…","""A03""","""BJ1896004""",30.0,"""10"""
"""bfmoalive108_P…","""A04""","""BJ1894345""",30.0,"""10"""
"""bfmoalive108_P…","""A05""","""BJ1897105""",10.0,"""10"""
"""bfmoalive108_P…","""A05""","""backfill""",20.0,"""100%"""
"""bfmoalive108_P…","""A06""","""BJ1897191""",2.5,"""10"""
"""bfmoalive108_P…","""A06""","""backfill""",27.5,"""100%"""


In [3]:
df2 = pl.read_csv('bfmoalive108_compoundlist.csv', separator=',')
df2

Compound,moa_group,target,moa,BatchID,SPECS_name,SPECS_moa,SPECS_target,tested_concentration_uM,grit_Jonne,flag_Jonne,grit_SPECS,Count_SPECS,mean_normalized_cellcount,Unified_cellcount,conc,cbkid,smiles,inchi,inkey
str,str,str,str,str,str,str,str,i64,f64,str,f64,f64,f64,f64,i64,str,str,str,str
"""API-1""","""AKT""","""Akt""","""Specific inhib…","""BJ1896216""","""API-1""","""AKT inhibitor""","""AKT1|AKT2|AKT3…",10,null,null,null,null,13.833333,13.833333,1,"""CBK308074""","""NC(=O)c1cn([C@…","""InChI=1S/C13H1…","""SPBWHPXCWJLQRU…"
"""AR-12""","""AKT""","""PDPK1""","""Inihbits PDK-1…","""BJ1896668""","""AR-12""","""phosphoinositi…","""PDPK1""",10,null,null,null,null,57.527778,57.527778,10,"""CBK290484""","""NCC(=O)Nc1ccc(…","""InChI=1S/C26H1…","""YULUCECVQOCQFQ…"
"""AZD5363""","""AKT""","""Akt""","""competitive pa…","""BJ1899041""","""AZD5363""","""AKT inhibitor""","""AKT1|AKT2|AKT3…",10,null,null,null,null,72.055556,72.055556,10,"""CBK290956""","""NC1(CCN(CC1)c1…","""InChI=1S/C21H2…","""JDUBGYFRJFOXQC…"
"""Buparlisib""","""AKT""","""PIK3CG""","""Inhibits upstr…","""BJ1898458""","""BKM120 (NVP-BK…","""PI3K inhibitor…","""PIK3CA|PIK3CG""",10,null,null,null,null,17.666667,17.666667,1,"""CBK278043""","""Nc1cc(c(cn1)-c…","""InChI=1S/C18H2…","""CWHUFRVAEUJCEF…"
"""GSK2110183""","""AKT""","""AKT1""","""competitive pa…","""BJ1897441""","""Afuresertib""","""AKT inhibitor""","""AKT1|AKT2|AKT3…",10,null,null,null,null,66.361111,66.361111,10,"""CBK303904""","""Cn1ncc(Cl)c1-c…","""InChI=1S/C18H1…","""AFJRDFWMXUECEW…"
"""Ipatasertib""","""AKT""","""Akt""","""competitive pa…","""BJ1898683""","""Ipatasertib (G…","""AKT inhibitor""","""AKT1|AKT2|AKT3…",10,null,null,null,null,77.777778,77.777778,10,"""CBK278103""","""CC(C)NC[C@@H](…","""InChI=1S/C24H3…","""GRZXWCHAXNAUHY…"
"""LY294002""","""AKT""","""PI3K""","""Inhibits upstr…","""BJ1894333""","""LY294002""","""DNA dependent …","""AKT1|CHEK1|GSK…",10,null,null,1.4,83.4,null,83.4,10,"""CBK041160""","""O=c1cc(oc2c(cc…","""InChI=1S/C19H1…","""CZQHHVNHHHRRDU…"
"""MK-2206""","""AKT""","""Akt""","""Specific inhib…","""BJ1898163""","""MK-2206 dihydr…","""AKT inhibitor""","""AKT1|AKT2|AKT3…",10,null,null,null,null,65.833333,65.833333,10,"""CBK277971""","""NC1(CCC1)c1ccc…","""InChI=1S/C25H2…","""ULDXWLCXEDXJGE…"
"""PI-103""","""AKT""","""PI3K""","""Inhibits PI3K,…","""BJ1895998""","""PI-103""","""mTOR inhibitor…","""MTOR|PIK3CA|PI…",10,null,null,4.4,43.2,null,43.2,3,"""CBK288319""","""Oc1cccc(c1)-c1…","""InChI=1S/C19H1…","""TUVCWJQQGGETHL…"


In [4]:
df2 = df2.with_columns(df2["BatchID"].alias("BatchID_right"))

In [5]:
df2

Compound,moa_group,target,moa,BatchID,SPECS_name,SPECS_moa,SPECS_target,tested_concentration_uM,grit_Jonne,flag_Jonne,grit_SPECS,Count_SPECS,mean_normalized_cellcount,Unified_cellcount,conc,cbkid,smiles,inchi,inkey,BatchID_right
str,str,str,str,str,str,str,str,i64,f64,str,f64,f64,f64,f64,i64,str,str,str,str,str
"""API-1""","""AKT""","""Akt""","""Specific inhib…","""BJ1896216""","""API-1""","""AKT inhibitor""","""AKT1|AKT2|AKT3…",10,null,null,null,null,13.833333,13.833333,1,"""CBK308074""","""NC(=O)c1cn([C@…","""InChI=1S/C13H1…","""SPBWHPXCWJLQRU…","""BJ1896216"""
"""AR-12""","""AKT""","""PDPK1""","""Inihbits PDK-1…","""BJ1896668""","""AR-12""","""phosphoinositi…","""PDPK1""",10,null,null,null,null,57.527778,57.527778,10,"""CBK290484""","""NCC(=O)Nc1ccc(…","""InChI=1S/C26H1…","""YULUCECVQOCQFQ…","""BJ1896668"""
"""AZD5363""","""AKT""","""Akt""","""competitive pa…","""BJ1899041""","""AZD5363""","""AKT inhibitor""","""AKT1|AKT2|AKT3…",10,null,null,null,null,72.055556,72.055556,10,"""CBK290956""","""NC1(CCN(CC1)c1…","""InChI=1S/C21H2…","""JDUBGYFRJFOXQC…","""BJ1899041"""
"""Buparlisib""","""AKT""","""PIK3CG""","""Inhibits upstr…","""BJ1898458""","""BKM120 (NVP-BK…","""PI3K inhibitor…","""PIK3CA|PIK3CG""",10,null,null,null,null,17.666667,17.666667,1,"""CBK278043""","""Nc1cc(c(cn1)-c…","""InChI=1S/C18H2…","""CWHUFRVAEUJCEF…","""BJ1898458"""
"""GSK2110183""","""AKT""","""AKT1""","""competitive pa…","""BJ1897441""","""Afuresertib""","""AKT inhibitor""","""AKT1|AKT2|AKT3…",10,null,null,null,null,66.361111,66.361111,10,"""CBK303904""","""Cn1ncc(Cl)c1-c…","""InChI=1S/C18H1…","""AFJRDFWMXUECEW…","""BJ1897441"""
"""Ipatasertib""","""AKT""","""Akt""","""competitive pa…","""BJ1898683""","""Ipatasertib (G…","""AKT inhibitor""","""AKT1|AKT2|AKT3…",10,null,null,null,null,77.777778,77.777778,10,"""CBK278103""","""CC(C)NC[C@@H](…","""InChI=1S/C24H3…","""GRZXWCHAXNAUHY…","""BJ1898683"""
"""LY294002""","""AKT""","""PI3K""","""Inhibits upstr…","""BJ1894333""","""LY294002""","""DNA dependent …","""AKT1|CHEK1|GSK…",10,null,null,1.4,83.4,null,83.4,10,"""CBK041160""","""O=c1cc(oc2c(cc…","""InChI=1S/C19H1…","""CZQHHVNHHHRRDU…","""BJ1894333"""
"""MK-2206""","""AKT""","""Akt""","""Specific inhib…","""BJ1898163""","""MK-2206 dihydr…","""AKT inhibitor""","""AKT1|AKT2|AKT3…",10,null,null,null,null,65.833333,65.833333,10,"""CBK277971""","""NC1(CCC1)c1ccc…","""InChI=1S/C25H2…","""ULDXWLCXEDXJGE…","""BJ1898163"""
"""PI-103""","""AKT""","""PI3K""","""Inhibits PI3K,…","""BJ1895998""","""PI-103""","""mTOR inhibitor…","""MTOR|PIK3CA|PI…",10,null,null,4.4,43.2,null,43.2,3,"""CBK288319""","""Oc1cccc(c1)-c1…","""InChI=1S/C19H1…","""TUVCWJQQGGETHL…","""BJ1895998"""


In [6]:
df3 = df1.join(df2, left_on="cbk_batch", right_on="BatchID", how="left")

In [7]:
df3

plate_name,well_position,cbk_batch,comp_vol_nL,stock_conc_mM,Compound,moa_group,target,moa,SPECS_name,SPECS_moa,SPECS_target,tested_concentration_uM,grit_Jonne,flag_Jonne,grit_SPECS,Count_SPECS,mean_normalized_cellcount,Unified_cellcount,conc,cbkid,smiles,inchi,inkey,BatchID_right
str,str,str,f64,str,str,str,str,str,str,str,str,i64,f64,str,f64,f64,f64,f64,i64,str,str,str,str,str
"""bfmoalive108_P…","""A01""","""backfill""",30.0,"""100%""",null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null
"""bfmoalive108_P…","""A02""","""BJ1894267""",10.0,"""10""","""Indibulin""","""TUB""",null,"""tubulin polyme…","""Indibulin""","""tubulin polyme…",null,null,null,null,4.370455,37.413827,null,null,3,null,null,null,null,"""BJ1894267"""
"""bfmoalive108_P…","""A02""","""backfill""",20.0,"""100%""",null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null
"""bfmoalive108_P…","""A03""","""BJ1896004""",30.0,"""10""","""RWJ-67657""","""MAPK""","""MAPK""","""undefined""","""RWJ-67657""","""MAP kinase inh…","""MAPK14|PTGS2""",10,null,null,null,null,82.75,82.75,10,"""CBK307989""","""OCCC#Cc1nc(c(-…","""InChI=1S/C27H2…","""QSUSKMBNZQHHPA…","""BJ1896004"""
"""bfmoalive108_P…","""A04""","""BJ1894345""",30.0,"""10""","""SB-203580""","""MAPK""","""MAPK14""","""undefined""","""SB-203580""","""p38 MAPK inhib…","""AKT1|ALOX5|CHE…",10,null,null,null,null,null,null,10,"""CBK041168""","""CS(=O)c1ccc(cc…","""InChI=1S/C21H1…","""CDMGBJANTYXAIV…","""BJ1894345"""
"""bfmoalive108_P…","""A05""","""BJ1897105""",10.0,"""10""","""LY2857785""","""CDK""","""CDK9""","""undefined""","""LY2857785""","""CDK inhibitor""","""CDK9""",10,null,null,null,null,21.861111,21.861111,3,"""CBK308713""","""CC(C)c1n(C)nc2…","""InChI=1S/C26H3…","""LHIUZPIDLZYPRL…","""BJ1897105"""
"""bfmoalive108_P…","""A05""","""backfill""",20.0,"""100%""",null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null
"""bfmoalive108_P…","""A06""","""BJ1897191""",2.5,"""10""","""Cobimetinib""","""MAPK""","""MAP2K1""","""undefined""","""Cobimetinib""","""MEK inhibitor""","""MAP2K1""",10,null,null,null,null,14.666667,14.666667,1,"""CBK303945""","""OC1(CN(C1)C(=O…","""InChI=1S/C21H2…","""BSMCAPRUBJMWDF…","""BJ1897191"""
"""bfmoalive108_P…","""A06""","""backfill""",27.5,"""100%""",null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null


In [8]:
df3.write_csv("bfmoadlive108_platelayout_with_metadata_v2.csv")

In [9]:
approx_unique_count = df1.select(pl.col('cbk_batch').approx_n_unique())

In [10]:
approx_unique_count

cbk_batch
u32
112


In [11]:
approx_unique_count = df2.select(pl.col('BatchID').approx_n_unique())

In [12]:
approx_unique_count

BatchID
u32
108


In [13]:
approx_unique_count = df3.select(pl.col('cbk_batch').approx_n_unique())

In [14]:
approx_unique_count

cbk_batch
u32
112
